In [1]:
import numpy as np
import pandas as pd
import pymongo as mongo
import re


import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, PCA, TruncatedSVD, LatentDirichletAllocation
from sklearn.utils.extmath import randomized_svd

from textblob import TextBlob, Word, WordList
from gensim import models, corpora, similarities, matutils

from sys import getsizeof
import psutil
import pickle
from multiprocessing import Pool

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
psutil.virtual_memory()

svmem(total=8589934592L, available=2985017344L, percent=65.2, used=6844125184L, free=1400344576L, active=3977486336L, inactive=1584672768L, wired=1281966080L)

In [3]:
pd.options.display.max_columns = 50

In [4]:
client = mongo.MongoClient()

In [5]:
client.database_names()

[u'admin', u'local', u'p4']

In [6]:
db = client.get_database('p4')

In [7]:
journals = db.collection_names()[:-1] # drop system.indexes...

In [8]:
print len(journals)
total = 0
for j in journals:
    total += db[j].count()

total

275


25349

In [9]:
# Fixing poor naming schema from before
for j in journals:
    coll = db.get_collection(j)
    coll.update_many({'Max_Topic_Name':'Attention??'}, {"$set":{'Max_Topic_Name':"Attention"}})
    coll.update_many({},{"$rename": {"Attention??":"Attention"}})

CountVectorizer()
#### input can equal a list of filenames to read (or file objects)

#### analyzer:
    - default is "word"
    - Extracts each interpreted "word" as features
    
#### The Analyzer is composed of 2 steps:

1) Preprocessing
    - This is the general string formatting that is done before tokenizing
    - lowercasing, removing accents
    - does not seem to include stemming/lemmatizing :-/??
    - though you may be able to pass in a function that does whatever you want...
2) Tokenizing
    - Creating a vocabulary based on what is considered to be a "word"
    - by default, all groups of 2 or more consecutive alpha-numeric characters are considered "words".
        * punctuation is ignored and always splits "words"
    - you can set the definition using token_pattern = regex...


#### ngram_range:
    - number of ngrams to consider for features
    - format is (min, max)
#### stop_words:
    - can be 'english' or list of stopwords
#### lowercase:
    - Boolean default is True
    
    
#### min_df and max_df:
    - set min and max occurences required for each ftr


In [10]:
def JournalData(j):
    collec = db.get_collection(j)    
    
    # find returns a curser which gets a list of dictionaries
    data = collec.find({})
    # in addition to our features include columns for Journal and Date
    ids = []
    journals = []
    dates = [] 
    text_data = []
    for i in range(data.count()):
        doc = data.next()
        ids.append(doc['_id'])
        dates.append(doc['date'])
        text_data.append(doc['text'])
        journals.append(j)
    return ids, journals, dates, text_data



In [11]:
ids = []
jnls = []
dates = []
text_data = []

In [12]:
# BUILD LISTS CONTAINING ALL JOURNAL_NAMES DATES AND ARTICLES

for j in journals:
#     if j in jnls:
#         continue
    i, j, d, t = JournalData(j)
    ids.extend(i)
    jnls.extend(j)
    dates.extend(d)
    text_data.extend(t)
    print(getsizeof(jnls))
    if getsizeof(jnls) > 1000000000:
        print('SHIT THATS BIG (THATS WHAT SHE SAID)')
        break

136
136
200
200
336
7824
7824
7824
11096
12896
12896
12896
12896
16008
16008
16008
16008
16008
18168
27312
27312
27312
27312
27312
27312
27312
41352
41352
41352
41352
41352
41352
41352
41352
41352
41352
48272
48272
48272
48272
48272
48272
48272
48272
48272
48272
48272
48272
48272
54504
54504
54504
54504
54504
54504
54504
54504
54504
54504
54504
54504
54504
54504
62800
62800
62800
62800
62800
62800
62800
62800
62800
62800
70872
70872
70872
70872
80360
80360
80360
80360
80360
80360
80360
80360
80360
80360
80360
80360
80360
80360
80360
80360
80360
80360
80360
80360
91904
91904
91904
91904
91904
91904
91904
91904
91904
91904
91904
91904
91904
91904
91904
91904
91904
91904
91904
91904
91904
103464
103464
103464
103464
103464
103464
103464
103464
103464
103464
103464
103464
103464
103464
103464
103464
103464
103464
103464
103464
103464
103464
103464
103464
117128
117128
117128
117128
117128
117128
117128
117128
117128
117128
117128
117128
117128
117128
117128
117128
117128
117128
117128
1171

In [13]:
print(len(ids), len(jnls), len(dates), len(text_data))

(25349, 25349, 25349, 25349)


## CLEANING TEXT DATA

In [14]:
body = re.compile(r'== ?[bB]ody ')

def clean_doc(doc):
    # needs textblob
    
    # GET RID OF NEWLINES
    doc = doc.replace('\n', ' ')
    
    # STRIP HEADING INFO
    # if there's no body section, get rid of it
    if not re.search(body, doc):
        return None
    mtch = re.search(body,doc)
    # strip anything before match
    doc = doc[mtch.start():]

    # LOWERCASE!
    doc = doc.lower()

    #SINGULARIZE
    # this seriously slows it down!
    #doc = ' '.join([Word(w).singularize() for w in doc.split()])
    return doc
        
    
    



In [15]:
idx = 0
cleaned = []

In [16]:
pool = Pool(4)
cleaned = pool.map(clean_doc, text_data)
pool.close()
len(cleaned)

25349

In [17]:
psutil.virtual_memory()

svmem(total=8589934592L, available=1318989824L, percent=84.6, used=3323179008L, free=344854528L, active=1193730048L, inactive=974135296L, wired=1155313664L)

In [18]:
# USE POOL - MUCH FASTER!
# Obviously this takes a while...
# Maybe too long
# len(text_data) # 25370


# for t in iter(text_data[idx:]):
#     print(idx)
#     cleaned.append(clean_doc(t))
#     idx+=1
# len(cleaned)

In [19]:
# REMOVE STOP WORDS

# stop_words = nltk.corpus.stopwords.words()
# len(stop_words) # 2781 ... i think has other languages need only english

# englishStopWords = None
# with open('csvs/EngStopWords.csv', 'r') as f:
#     englishStopWords = f.read().split(', ')
    
    
# len(englishStopWords) # 174 better


In [20]:
with open('WranglingPickles/cleaned.pkl', 'wb') as f:
    pickle.dump(cleaned, f, 2)

In [20]:
with open('WranglingPickles/cleaned.pkl', 'rb') as f:
    cleaned = pickle.load(f)

In [21]:
len(cleaned)

25349

In [22]:
# Join With Jrnl Name and Date

Joined = list(zip(ids,jnls, dates, range(len(cleaned))))



# REMOVE NULLS
Joined = [a for a in Joined if cleaned[a[3]]]
cleaned = list(filter(None, cleaned))

print(len(cleaned), len(Joined))

(25306, 25306)


## TFIDFVECTORIZE

TFIDF to distinguish more distinct words for clearer topics!

In [33]:
stopwords = '''a, about, above, after, again, against, all, am, an, and, any, are, aren't, as, at, be, because, been, before, being, below, between, both, but, by, can't, cannot, could, couldn't, did, didn't, do, does, doesn't, doing, don't, down, during, each, few, for, from, further, had, hadn't, has, hasn't, have, haven't, having, he, he'd, he'll, he's, her, here, here's, hers, herself, him, himself, his, how, how's, i, i'd, i'll, i'm, i've, if, in, into, is, isn't, it, it's, its, itself, let's, me, more, most, mustn't, my, myself, no, nor, not, of, off, on, once, only, or, other, ought, our, ours, ourselves, out, over, own, same, shan't, she, she'd, she'll, she's, should, shouldn't, so, some, such, than, that, that's, the, their, theirs, them, themselves, then, there, there's, these, they, they'd, they'll, they're, they've, this, those, through, to, too, under, until, up, very, was, wasn't, we, we'd, we'll, we're, we've, were, weren't, what, what's, when, when's, where, where's, which, while, who, who's, whom, why, why's, with, won't, would, wouldn't, you, you'd, you'll, you're, you've, your, yours, yourself, yourselves'''

stopwords = stopwords.split(',')
stopwords = [a.strip() for a in stopwords]

In [24]:
print(len(cleaned)) # 25327



cvectr = TfidfVectorizer(stop_words=stopwords, min_df=10, max_df = 0.5)
cvectr.fit(cleaned)


25306


TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.5, max_features=None, min_df=10,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing'..."wouldn't", 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves'],
        strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [34]:
len(stopwords)

174

In [30]:
len([word for word in cvectr.get_feature_names() if any(char.isdigit() for char in word)])


24633

In [35]:
stopwords.extend([n for n in cvectr.get_feature_names() if n <'a'])
stopwords.extend([n for n in cvectr.get_feature_names() if n >'z'])

len(stopwords)

21218

In [36]:
stopwords = list(set(stopwords))
len(stopwords)

21218

In [37]:
cvectr = TfidfVectorizer(stop_words=stopwords, min_df=10, max_df = 0.5)
txt_ftrs = cvectr.fit_transform(cleaned)


txt_ftrs

<25306x105463 sparse matrix of type '<type 'numpy.float64'>'
	with 23314348 stored elements in Compressed Sparse Row format>

In [38]:
wordDic = {d[1]: d[0] for d in cvectr.vocabulary_.items()}

print(len(wordDic))
print(max(wordDic.keys()))

105463
105462


## THINGS TO TRY:

#### Summarize
    - Apply PCA, SVD, NMF, LDA
    - Interpret results... aka grouping?? how?
    

#### Features:
    - POS
    - Somehow incorporate Parse Trees?
    
    
#### Beyond:
    - Word2vec
    - Heirarchical Clustering

#### PCA

In [110]:
# PCA Doesn't work cuz it doesn't accept sparse

#### LDA

In [111]:
# MEMORY ERROR

#### SVD

In [112]:
# GOING WITH NMF BECAUSE SVD TOPICS ARE CRUMMY

#### WHAT I SEE:
    - Remove Plurals
    - Adjust for Tenses
    - Can we fuse numbers/units for measurements?
    - Eliminate Random Numbers

#### NMF

In [52]:
nmf = NMF(init = 'nndsvd', n_components=15)

nmf.fit(txt_ftrs)

NMF(alpha=0.0, beta=1, eta=0.1, init='nndsvd', l1_ratio=0.0, max_iter=200,
  n_components=15, nls_max_iter=2000, random_state=None, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)

In [53]:
nmf.components_.shape

(15, 105463)

In [54]:
# get top 25 significant words for each component
top25 = {}

for i in range(len(nmf.components_)):# for each component
    # sort the words in the components by importance, but grab the original indices
    idxs = np.argsort(nmf.components_[i])[-1:-26:-1]
    # use idxs to extract corresponding words from wordDic
    top25['ftr_'+str(i)] = [wordDic[x] for x in idxs]

In [55]:
top25

{'ftr_0': [u'schizophrenia',
  u'depression',
  u'disorder',
  u'psychiatry',
  u'bipolar',
  u'symptoms',
  u'anxiety',
  u'depressive',
  u'disorders',
  u'psychiatric',
  u'antipsychotic',
  u'placebo',
  u'antidepressant',
  u'social',
  u'mood',
  u'adhd',
  u'stress',
  u'risk',
  u'health',
  u'mdd',
  u'mental',
  u'cognitive',
  u'antipsychotics',
  u'illness',
  u'drug'],
 'ftr_1': [u'cells',
  u'mice',
  u'cell',
  u'expression',
  u'neurons',
  u'protein',
  u'receptor',
  u'fig',
  u'rats',
  u'gene',
  u'mouse',
  u'induced',
  u'receptors',
  u'rat',
  u'stress',
  u'genes',
  u'animals',
  u'neuronal',
  u'signaling',
  u'astrocytes',
  u'microglia',
  u'proteins',
  u'activation',
  u'neurosci',
  u'anti'],
 'ftr_10': [u'tumor',
  u'tumors',
  u'resection',
  u'meningiomas',
  u'meningioma',
  u'cancer',
  u'metastases',
  u'grade',
  u'lesion',
  u'malignant',
  u'surgery',
  u'radiation',
  u'glioblastoma',
  u'surgical',
  u'radiotherapy',
  u'mass',
  u'metastasis'

In [56]:
text_nmf = pd.DataFrame(nmf.transform(txt_ftrs), columns = ['ftr_' + str(i) for i in range(len(top25.keys()))])
getsizeof(text_nmf) # 3mb
text_nmf.head(10)


,ftr_0,ftr_1,ftr_2,ftr_3,ftr_4,ftr_5,ftr_6,ftr_7,ftr_8,ftr_9,ftr_10,ftr_11,ftr_12,ftr_13,ftr_14
0,0.010591,0.002359,0.000000,0.001326,0.000630,0.005426,0.001974,0.000697,0.000486,0.002965,0.000000,0.002464,0.007804,0.004245,0.00000
1,0.000000,0.061366,0.000000,0.000000,0.000000,0.000000,0.000000,0.000282,0.000000,0.000000,0.000000,0.000000,0.000207,0.004653,0.00000
2,0.021576,0.017588,0.000000,0.000000,0.000000,0.004652,0.000000,0.000000,0.007767,0.002614,0.000000,0.005984,0.012790,0.000000,0.00000
3,0.023221,0.023862,0.000000,0.000000,0.000000,0.004107,0.000000,0.000000,0.004684,0.002792,0.000000,0.005859,0.017948,0.000000,0.00000
4,0.002251,0.024278,0.000000,0.004714,0.000000,0.004019,0.000000,0.001201,0.011214,0.085156,0.000000,0.001288,0.007382,0.000763,0.00000
5,0.000000,0.057558,0.001053,0.068055,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
6,0.000000,0.054796,0.001641,0.015876,0.000805,0.000000,0.000000,0.002154,0.000000,0.000000,0.008308,0.000000,0.000000,0.000000,0.00000
7,0.005659,0.001131,0.000000,0.000000,0.008524,0.007919,0.037048,0.000000,0.047169,0.000000,0.007529,0.002659,0.000000,0.001837,0.00000
8,0.000000,0.000000,0.001868,0.004783,0.004195,0.000000,0.001778,0.000000,0.014916,0.000000,0.000858,0.000000,0.000000,0.000000,0.08237
9,0.000000,0.008627,0.000000,0.000000,0.000000,0.000000,0.000000,0.000895,0.000000,0.000000,0.096124,0.000000,0.000000,0.000000,0.00000


In [57]:
components_df = pd.DataFrame(top25)

In [58]:
components_df = components_df[[u'ftr_0', u'ftr_1',u'ftr_2', u'ftr_3', u'ftr_4', u'ftr_5', u'ftr_6', u'ftr_7', u'ftr_8',u'ftr_9',
u'ftr_10', u'ftr_11', u'ftr_12', u'ftr_13',u'ftr_14']]

In [59]:
components_df

,ftr_0,ftr_1,ftr_2,ftr_3,ftr_4,ftr_5,ftr_6,ftr_7,ftr_8,ftr_9,ftr_10,ftr_11,ftr_12,ftr_13,ftr_14
0,schizophrenia,cells,aneurysm,neurons,spinal,stroke,epilepsy,usepackage,sclerosis,sleep,tumor,migraine,task,alzheimer,pd
1,depression,mice,aneurysms,spike,spine,ischemic,seizure,document,mri,rem,tumors,pain,cortex,ad,parkinson
2,disorder,cell,artery,synaptic,cord,acute,seizures,oddsidemargin,csf,insomnia,resection,headache,memory,dementia,dbs
3,psychiatry,expression,endovascular,firing,cervical,risk,eeg,wasysym,syndrome,circadian,meningiomas,ibs,visual,amyloid,stimulation
4,bipolar,neurons,stent,neuron,surgery,thrombolysis,children,upgreek,ms,rls,meningioma,bowel,connectivity,cognitive,levodopa
5,symptoms,protein,carotid,network,lumbar,cerebral,epileptic,amsfonts,lesions,apnea,cancer,irritable,fmri,aβ,motor
6,anxiety,receptor,intracranial,input,injury,artery,epilepsia,amssymb,nerve,osa,metastases,aura,participants,tau,dopamine
7,depressive,fig,angiography,neural,pain,carotid,aeds,mathrsfs,encephalitis,night,grade,reflux,processing,mci,stn
8,disorders,rats,cerebral,spikes,hematoma,nihss,temporal,amsmath,diagnosis,sleepiness,lesion,symptoms,regions,memory,tremor
9,psychiatric,gene,embolization,stimulus,disc,infarction,ictal,documentclass,matter,wake,malignant,cephalalgia,stimuli,impairment,synuclein


In [60]:
# text_data, cleaned, cvectr, nmf, text_nmf, components_df
with open('WranglingPickles/text_data.pkl','wb') as f:
    pickle.dump(text_data, f, 2)
    
with open('WranglingPickles/cleaned.pkl','wb') as f:
    pickle.dump(cleaned, f, 2)
    
with open('WranglingPickles/cvectr.pkl','wb') as f:
    pickle.dump(cvectr, f, 2)
    
with open('WranglingPickles/nmf.pkl','wb') as f:
    pickle.dump(nmf, f, 2)
    
with open('WranglingPickles/text_nmf.pkl','wb') as f:
    pickle.dump(text_nmf, f, 2)
    
with open('WranglingPickles/components_df.pkl','wb') as f:
    pickle.dump(components_df, f, 2)
    
with open('WranglingPickles/Joined.pkl','wb') as f:
    pickle.dump(Joined, f, 2)